In [1]:
import numpy as np
from pathlib import Path
from PIL import Image
from torch.utils.data import DataLoader, random_split
import torch
from torch import optim
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import logging
from evaluate import evaluate
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import OrderedDict

#from model.unet.unet_model import UNet
#from model.segnet.segnet_model import SegNet
#from torchvision.models.segmentation import deeplabv3_resnet101 as DeepLabv3
from model.ensemblenet_model import EnsembleNet


from utils.dice_score import dice_loss
from utils.data_load import KittiDataset
from torchsummaryX import summary

In [2]:
Val_Percent = 0.3
Scale_Percent = 1.0
Batch_Size = 8
learning_rate = 0.0001
Pin_Memory = False
epochs = 10

#Image_Size = [384, 1242]
Image_Size = [384, 1216]
#Image_Size = [384,384]
Gradient_Clipping = 1.0

#Num_Class = 31
#Num_Class = 21
Num_Class = 2
Num_Channel = 3
amp = True


Img_Path =  'data/data_road/training/image_2'
Mask_Path =  'data/data_road/training/semantic'

save_checkpoint = False
checkpoint_dir = '../trained'
batch_size = Batch_Size

In [3]:
dirImg = Path(Img_Path)
dirMask = Path(Mask_Path)

dir_checkpoint = Path(checkpoint_dir)

In [4]:
train_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        #A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        #A.RandomBrightnessContrast(p=0.5),
        #A.RandomGamma(p=0.5),
        #A.RandomSnow(p=0.5),
        #A.RandomRain(p=0.5),
        #A.RandomFog(p=0.5),
        #A.RandomSunFlare(p=0.5),
        A.RandomShadow(p=0.5),
        #A.RandomToneCurve(p=0.5),
        #A.GaussNoise(p=0.5),
        #A.Emboss(p=0.5),  # IAAEmboss 대신 Emboss 사용
        #A.Perspective(p=0.5),  # IAAPerspective 대신 Perspective 사용
        #A.CLAHE(p=0.5)
])

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
datasets =  KittiDataset(dirImg, dirMask, Image_Size, Scale_Percent)
#datasets =  KittiDataset(dirImg, dirMask, Image_Size, Scale_Percent, train_transform)
n_val = int(len(datasets) * Val_Percent)
n_train = len(datasets) - n_val
train_set, val_set = random_split(datasets, [n_train, n_val], generator=torch.Generator().manual_seed(0))

loader_args = dict(batch_size=Batch_Size, num_workers= os.cpu_count(), pin_memory=Pin_Memory)
train_loader = DataLoader(train_set, shuffle=True, drop_last = True, **loader_args)
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 289/289 [00:00<00:00, 817.68it/s]


In [7]:
model = EnsembleNet('deeplabv3', Num_Channel, Num_Class)
model = model.to(memory_format=torch.channels_last, device = device)

/home/user1/anaconda3/envs/ksh/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/user1/anaconda3/envs/ksh/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
# 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-8)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
criterion = nn.CrossEntropyLoss()
global_step = 0

In [9]:
valScore_list = []
TrainLoss_list = []
# 5. Begin training
for epoch in range(1, epochs + 1):
    model.train()
    epoch_loss = 0
    with tqdm(total=n_train, desc=f'Epoch {epoch}/{epochs}', unit='img') as pbar:
        for batch in train_loader:
            images, true_masks = batch['image'], batch['mask']

            images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            true_masks = true_masks.to(device=device, dtype=torch.long)

            with torch.autocast(device.type if device.type != 'mps' else 'cpu', enabled=amp):
                masks_pred = model(images)
                if isinstance(masks_pred, OrderedDict):
                    masks_pred = masks_pred['out']
                             
                try:
                    mn_cls = model.n_classes
                except:
                    mn_cls = model.classifier[-1].out_channels
                    
                
                if mn_cls == 1:
                    
                    squ_masks_pred = masks_pred.squeeze(1)
                    loss = criterion(squ_masks_pred, true_masks.float())
                    loss += dice_loss(F.sigmoid(squ_masks_pred), true_masks.float(), multiclass=False)
                    
                    #loss = criterion(masks_pred.squeeze(1), true_masks.float())
                    #loss += dice_loss(masks_pred, true_masks, multiclass=False)
                    
                else:
                    loss = criterion(masks_pred, true_masks)
                    loss += dice_loss(
                        F.softmax(masks_pred, dim=1).float(),
                        F.one_hot(true_masks, mn_cls).permute(0, 3, 1, 2).float(),
                        multiclass=True
                    )
                

            optimizer.zero_grad(set_to_none=True)
            grad_scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), Gradient_Clipping)
            grad_scaler.step(optimizer)
            grad_scaler.update()

            pbar.update(images.shape[0])
            global_step += 1
            epoch_loss += loss.item()


            # Evaluation round
            division_step = (n_train // (5 * batch_size))
            if division_step > 0:
                if global_step % division_step == 0:

                    val_score = evaluate(model, val_loader, device, amp)
                    
                    scheduler.step(val_score)
                    valScore_list.append(val_score)
                    TrainLoss_list.append(loss)
                    

                    #logging.info('Validation Dice score: {}'.format(val_score))
                    print('Training Dice Loss: {}'.format(loss))
                    print('Validation Dice score: {}'.format(val_score))
                                
            
    if save_checkpoint:
        Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), str(dir_checkpoint / 'checkpoint_epoch{}.pth'.format(epoch + 1)))

Epoch 1/10:  20%|██████████████████████████████████████████████▎                                                                                                                                                                                            | 40/203 [00:05<00:13, 12.43img/s]

Training Dice Loss: 0.391745001077652
Validation Dice score: 0.4460718333721161


Epoch 1/10:  39%|████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                                              | 80/203 [00:09<00:07, 15.46img/s]

Training Dice Loss: 0.13671737909317017
Validation Dice score: 0.7065948247909546


Epoch 1/10:  59%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                               | 120/203 [00:13<00:05, 15.86img/s]

...

Training Dice Loss: 0.02460905909538269
Validation Dice score: 0.9644418954849243


Epoch 10/10:  79%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                 | 160/203 [00:15<00:02, 15.64img/s]

Training Dice Loss: 0.022302305325865746
Validation Dice score: 0.963493287563324


Epoch 10/10:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:21<00:00,  9.20img/s]

Training Dice Loss: 0.02116331085562706
Validation Dice score: 0.963901937007904


In [11]:
valScore_list

[tensor(0.4461, device='cuda:0'),
 tensor(0.7066, device='cuda:0'),
 tensor(0.7842, device='cuda:0'),
 tensor(0.7259, device='cuda:0'),
 tensor(0.7876, device='cuda:0'),
 tensor(0.8873, device='cuda:0'),
 tensor(0.9105, device='cuda:0'),
 tensor(0.8723, device='cuda:0'),
 tensor(0.9304, device='cuda:0'),
 tensor(0.9438, device='cuda:0'),
 tensor(0.9486, device='cuda:0'),
 tensor(0.9297, device='cuda:0'),
 tensor(0.9496, device='cuda:0'),
 tensor(0.9369, device='cuda:0'),
 tensor(0.9490, device='cuda:0'),
 tensor(0.9288, device='cuda:0'),
 tensor(0.9442, device='cuda:0'),
 tensor(0.9550, device='cuda:0'),
 tensor(0.9571, device='cuda:0'),
 tensor(0.9579, device='cuda:0'),
 tensor(0.9598, device='cuda:0'),
 tensor(0.9610, device='cuda:0'),
 tensor(0.9620, device='cuda:0'),
 tensor(0.9627, device='cuda:0'),
 tensor(0.9636, device='cuda:0'),
 tensor(0.9634, device='cuda:0'),
 tensor(0.9643, device='cuda:0'),
 tensor(0.9641, device='cuda:0'),
 tensor(0.9634, device='cuda:0'),
 tensor(0.9640

In [12]:
TrainLoss_list

[tensor(0.3917, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.1367, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.1151, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.2557, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.1054, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0902, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0724, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0550, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0648, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0617, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0639, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0401, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0525, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0346, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0353, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0692, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0477, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0306

In [10]:
dd = masks_pred.squeeze(1).squeeze(1)

In [15]:
dd2 = masks_pred.squeeze(1)

In [15]:
F.sigmoid(masks_pred).size()

torch.Size([16, 1, 384, 1216])

In [16]:
true_masks.shape

torch.Size([16, 384, 1216])

In [11]:
F.sigmoid(squ_masks_pred).shape

torch.Size([16, 384, 1216])